<a href="https://colab.research.google.com/github/efitzgerald763/Blood_brain_models/blob/main/ENSG00000096060_pediction_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connects to your Google Drive so you can import files
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

file_path = '/content/drive/My Drive/Colab Notebooks/Blood_brain_pred/ENSG00000096060_blood_brain.csv'
data = pd.read_csv(file_path)

# Set the index to the first column
data.set_index(data.columns[0], inplace=True)

Mounted at /content/drive


In [2]:
# Transpose the dataframe so each gene is a feature and each sample is a column
data_transposed = data.T

data_transposed.sample(4)

# Separate the target variable
target_row = 'ENSG00000096060'
y = data_transposed[target_row]
X = data_transposed.drop(columns=[target_row])

# Check the shapes to ensure they are as expected
print(X.shape)
print(y.shape)

(81, 18706)
(81,)


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the VAE model
original_dim = X_scaled.shape[1]
intermediate_dim = 64
latent_dim = 10

# Encoder
inputs = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='linear')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# VAE model
vae = Model(inputs, x_decoded_mean)

# Loss function
xent_loss = original_dim * tf.keras.losses.mean_squared_error(inputs, x_decoded_mean)
kl_loss = -0.5 * K.mean(z_log_var - K.square(z_mean) - K.exp(z_log_var) + 1, axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

# Train the VAE
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)
vae.fit(X_train, epochs=500, batch_size=10, validation_data=(X_test, None))



Epoch 1/500
7/7 [==============================] - 2s 71ms/step - loss: 18614.5254 - val_loss: 20150.2422
Epoch 2/500
7/7 [==============================] - 0s 36ms/step - loss: 18063.4434 - val_loss: 19957.9297
Epoch 3/500
7/7 [==============================] - 0s 37ms/step - loss: 18225.2168 - val_loss: 19971.8730
Epoch 4/500
7/7 [==============================] - 0s 36ms/step - loss: 18283.9824 - val_loss: 20004.1406
Epoch 5/500
7/7 [==============================] - 0s 36ms/step - loss: 17921.2461 - val_loss: 19795.8418
Epoch 6/500
7/7 [==============================] - 0s 39ms/step - loss: 17670.0742 - val_loss: 19634.6602
Epoch 7/500
7/7 [==============================] - 0s 38ms/step - loss: 17339.5137 - val_loss: 19443.2383
Epoch 8/500
7/7 [==============================] - 0s 36ms/step - loss: 16909.3789 - val_loss: 19147.9180
Epoch 9/500
7/7 [==============================] - 0s 41ms/step - loss: 16674.5508 - val_loss: 18839.3945
Epoch 10/500
7/7 [============================

In [6]:
# Define the encoder model to get the latent representation
encoder = Model(inputs, z_mean)
X_latent = encoder.predict(X)
print(X_latent.shape)  # Should be (number_of_samples, latent_dim)


3/3 [==============================] - 0s 19ms/step
(81, 10)


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Split the data
X_train_latent, X_test_latent, y_train, y_test = train_test_split(X_latent, y, test_size=0.2, random_state=42)

# Train the linear regression model
regressor = LinearRegression()
regressor.fit(X_train_latent, y_train)

# Predict and evaluate
y_pred = regressor.predict(X_test_latent)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error:", mse)
print(f'R^2 Score: {r2}')


Mean Squared Error: 0.4537696022909348
R^2 Score: 0.16205887298508959
